In [10]:
import pandas as pd
import numpy as np
import static_pulls as s_pulls 
import datetime as dt
import time

"""import "static_pulls" and then call the method 
"static_pulls.static_request(#today's date#, #list of unique ISINs from column AC#, ['OPID'], 
datatype = #placeholder#, write_df = False)"
setting write_df to false returns a pandas dataframe to the variable rather than writing a csv file to a folder
"""

def amendments(amend_df, input_df, max_year):
    for index, row in amend_df.iterrows():
        last_yr = row['last year']
        index_name = row['Taxonomy']
        isin = row['Identifier']
        if last_yr == 'nan':
            input_df.loc[input_df['ISIN'] == isin, index_name] = 0
        else:
            year_ls = list(range(int(float(last_yr) + 2), max_year + 1))
            input_df.loc[(input_df['Snapshot Date (Year)'].isin(year_ls)) & (input_df['ISIN'] == isin), index_name] = 0
    
    return input_df

def create_exp_table(df_new, row_labels):
    """Create the empty time series table"""
    unique_perid = sorted(df_new['Perm ID'].unique())
    columns = ['Row Labels'] + unique_perid
    exp_table = pd.DataFrame(columns=columns)
    exp_table['Row Labels'] = pd.Series(row_labels)
    exp_table.set_index('Row Labels', inplace=True)   
    return exp_table

def create_timeseries(df):
    """Create the final time series by removing the last row and adding Year Used column"""
    df_cleaned = df.iloc[:df.shape[0] - 1, :]
    year_used = df_cleaned.index.astype(int) + 1
    year_used = year_used.tolist()
    df_op = df_cleaned.copy()
    df_op.insert(loc=0, column='Year Used', value=year_used)
    return df_op

def exp_or_code(choice):
    """This method allows users to choose between max exposure or its sector code"""
    if choice == 1:
        return 'Max Weight'
    else:
        return 'Max Weighted Sector Code'

def fill_NA_opid(isin_opid_map):
    """This method aims to fill NaN value of Perm ID by its corresponding ISIN"""
    temp = isin_opid_map.copy()
    for key, val in temp.items():
        if val == 'NA':
            temp[key] = key
    return temp
    
def fill_exposure(df_tax_1, PG_bio_revo, exp_code):
    """Fill max exposure or the sector code into the time series table"""
    for index, row in df_tax_1.iterrows():
        year = row['fiscal_year']
        perid = row['Perm ID']
        PG_bio_revo.loc[year, perid] = row[exp_code]
    return PG_bio_revo

def front_back_fill(df):
    """Front fill and back fill the time series of each thematic index"""
    output = df.ffill(axis=0)
    output.fillna(method='bfill', inplace=True)
    output.fillna(0, inplace=True)
    return output

def get_rbr_id(df_rbr, taxonomy_ls, level_ls):
    """This method aims to get the rbr id of the selected taxonomy and levels.
    The input include a list of desired taxonomy and a list of desired levels.
    taxonomy_ls is a list of strings, and level_ls is a list of integers."""
    sub_df = df_rbr.loc[(df_rbr.taxonomy.isin(taxonomy_ls)) & df_rbr.level.isin(level_ls)]
    return sub_df.rbr_id.values

def get_rbr_id_map(df_rbr, taxonomy_ls, level_ls):
    """Obtain the dictionary of key as rbr id and value as rbr name"""
    sub_df = df_rbr.loc[(df_rbr.taxonomy.isin(taxonomy_ls)) & df_rbr.level.isin(level_ls)]
    rbr_id_map = dict(zip(sub_df.rbr_id.apply(str), sub_df.name))
    return rbr_id_map

def get_rbr_data(df_rbr_data, rbr_id):
    """This method aims to get columns correspondent to the selected rbr id, and then clean 
    the data by taking the absolute value and replace 0 by 100000.
    The rbr_id is a numpy array of integers. The df_rbr_data is the rbr data package."""
    output = []
    for x in rbr_id:
        col = str(x)
        if col in df2.columns.tolist():
            output.append(df2[col])
        else:
            print(col, ' -- ID Not Found in rbr data')
    new_df = pd.concat(output, axis=1).abs()
    new_df.replace(0, 100000, inplace=True)
    return new_df

def get_index_weight(df5_key, df_timeseries):
    """This method aims to obtain the columns of the time series table"""
    output = [np.nan] * len(df5_key)
    opid_list = df_timeseries.columns[1:]
    for index, row in df5_key.iterrows():
        year = row['Snapshot Date (Year)']
        opid = str(row['Final ID'])
        if opid in opid_list:
            val = df_timeseries[df_timeseries['Year Used'] == year].loc[:,opid].values[0]
            output[index] = val
    return output

def get_max_weight_code(get_rbr_data_op, rbr_id_map):
    """This method aims to obtain the column with max weight code."""
    max_weight = pd.DataFrame(get_rbr_data_op.max(axis=1), columns=['Max Weight'])
    max_weight_code = pd.DataFrame(get_rbr_data_op.idxmax(axis='columns'), columns=['Max Weighted Sector Code'])
    new_df = pd.concat([get_rbr_data_op, max_weight_code, max_weight], axis=1)
    new_df['Max Weighted Sector Name'] = new_df['Max Weighted Sector Code'].map(rbr_id_map)
    return new_df

def get_row_label(df_tax_1):
    """Obtain the index of the time series table"""
    row_labels = sorted(df_tax_1[df_tax_1['fiscal_year'].notna()]['fiscal_year'].unique().astype(int))
    row_labels += [np.nan]
    return row_labels

def max_weight_code_name(index_list, level_1, df_fix, df1, df2, df3, df5, choice):
    """Obtain the dataframe ready for screening and weighting"""
    # level_1 is a boolean variable which determines whether we focus on level 1 or the level with highest exp
    output = df5.copy()
    df5_key = df5[['Snapshot Date (Year)', 'Final ID']]
    
    for i in range(len(index_list)):
        index_name = index_list[i]
        tax_l = [index_name]
        if level_1:
            level_l = [1] #####
        else:
            level_l = df3[df3['Taxonomy Name Matched to Map'] == index_name]['Taxonomy Level Used'].values.tolist()
            print(level_l)
        print(tax_l)
        rbr_id = get_rbr_id(df1, tax_l, level_l)
        print(rbr_id)
        rbr_id_map = get_rbr_id_map(df1, tax_l, level_l)
        print(rbr_id_map)
        
        ############
        """
        if index_name == 'Syntax Thematic Digital Health Index':
            rbr_id = rbr_id[rbr_id != 46059]
            rbr_id_map.pop(str(46059))
            
            print(rbr_id)
            print(rbr_id_map)"""
        ############
        
        rbr_data = get_rbr_data(df2, rbr_id)

        max_weight_code = get_max_weight_code(rbr_data, rbr_id_map)
        
        df_new = pd.concat([df_fix, max_weight_code], axis=1)
        
        exp_code = exp_or_code(choice)
        
        sub_df = df_new[df_new[exp_code].notna()][['Perm ID', 'fiscal_year', exp_code]] # extract non-NaN
        row_labels = get_row_label(df_new)
        exp_table = create_exp_table(df_new, row_labels)
        
        exp_filled = fill_exposure(df_new, exp_table, exp_code) # fill exposure or code table
        exp_filled_fb = front_back_fill(exp_filled) # front and back fill
        exp_timeseries = create_timeseries(exp_filled_fb)

        opid_list = exp_timeseries.columns[1:]
        col_data = get_index_weight(df5_key, exp_timeseries)
        #print(col_data[:10])
        col_name = index_abv_list[i]
        output.loc[:, col_name] = col_data # add a new column with column name equal to the index name
        
    return output   

# change gvkey column from number to string
df2 = pd.read_excel('no_filter_rbr_data_package_2023-06-13_v3.xlsx', 
                    converters={'gvkey': str})
df1 = pd.read_excel('rbr_id_map_2023-06-12_no_filter.xlsx')
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
df3 = pd.read_excel('taxonomy_name_level4.xlsx')
df5 = pd.read_excel('thematic_initial7_snapshot.xlsx')
amend_df = pd.read_excel('classification_amend.xlsx')
saas = pd.read_excel('SaaS.xlsx')

df5['OPID'] = df5['OPID'].astype(str)
df5['Final ID'] = df5['Final ID'].astype(str)
df5['DSCD'] = df5['DSCD'].astype(str)
df5['ISIN'] = df5['ISIN'].astype(str)

# Front Fill gvkey due to the excel format
df2['gvkey'].fillna(method='ffill', inplace=True)

# Front Fill fiscal_year after groupby gvkey
# df2.update(df2.groupby('gvkey')['fiscal_year'].apply(lambda x: x.ffill()))

# Sort df2 by gvkey and fiscal_date
df2 = df2.sort_values(['gvkey', 'fiscal_date'])

# Drop duplicates of gvkey and fiscal_year and keep the last one
df2.drop_duplicates(subset=['gvkey', 'fiscal_year'], keep='last', inplace=True)
df2.reset_index(inplace=True, drop=True)

# fill NAN in fiscal_year by latest year
df2['fiscal_year'].fillna(2022, inplace=True)

# create df_fix by extracting the first 4 columns
df_fix = df2.iloc[:, :4]

# Fill the empty ISIN by its corresponding gvkey
df_fix['isin'].fillna(df_fix['gvkey'], inplace=True)

# Add Perm ID based on ISIN
ticks = df_fix['isin'].unique()
asofdate = dt.date.today().strftime('%Y-%m-%d')

# Run static_request to pull OPID
"""USE mappings from thematic_initial"""
#get_stats = s_pulls.static_request(asofdate, ticks, ['OPID'], 'placeholder', write_df = False)
#isin_opid_map = dict(zip(get_stats.Instrument, get_stats.Value))
isin_opid_map = dict(zip(df5['ISIN'], df5['Final ID']))

# Fill NaN Perm ID
isin_opid_map2 = fill_NA_opid(isin_opid_map)
df_fix.insert(0, 'Perm ID', df_fix['isin'].map(isin_opid_map2))
df_fix['Perm ID'].fillna(df_fix['isin'], inplace=True)

print('The number of unique Perm ID:', df_fix['Perm ID'].nunique())

index_list = df3['Taxonomy Name Matched to Map'].values.tolist()
print(index_list)

index_abv_list = ['Cybersecurity', 'Cloud', 'IoT', 'E-Commerce', 'Battery', 'Bio Revolution', 'Clean Energy', 
                  'SaaS', 'Defensive', 'Infrastructure', 'Real Asset', 'Inflation','Digital Health']



The number of unique Perm ID: 11157
['Syntax Thematic Cybersecurity Index', 'Syntax Thematic Cloud Compute Index', 'Syntax Thematic IoT Index', 'Syntax Thematic E-commerce Index', 'Syntax Thematic Battery Index', 'Syntax Thematic Bio Revolution Index', 'Syntax Thematic Clean Energy Index', 'Syntax Thematic Software-as-a-Service (SaaS) Index', 'Syntax Thematic Defensive Index', 'Syntax Thematic Infrastructure Index', 'Syntax Thematic Real Asset Index', 'Syntax Thematic Inflation Index', 'Syntax Thematic Digital Health Index']


In [11]:
def max_weight_code_name(index_list, level_1, df_fix, df1, df2, df3, df5, choice):
    """Obtain the dataframe ready for screening and weighting"""
    # level_1 is a boolean variable which determines whether we focus on level 1 or the level with highest exp
    output = df5.copy()
    df5_key = df5[['Snapshot Date (Year)', 'Final ID']]
    df5_key['Snapshot Date (Year)'] = df5_key['Snapshot Date (Year)'].astype(int)
    
    for i in range(len(index_list)):
        index_name = index_list[i]
        tax_l = [index_name]
        if level_1:
            level_l = [1]
        else:
            level_l = df3[df3['Taxonomy Name Matched to Map'] == index_name]['Taxonomy Level Used'].values.tolist()
            print(level_l)
        print(tax_l)
        rbr_id = get_rbr_id(df1, tax_l, level_l)
        print(rbr_id)
        rbr_id_map = get_rbr_id_map(df1, tax_l, level_l)
        print(rbr_id_map)
        
        ############
        """
        if index_name == 'Syntax Thematic Digital Health Index':
            rbr_id = rbr_id[rbr_id != 46059]
            rbr_id_map.pop(str(46059))
            
            print(rbr_id)
            print(rbr_id_map)"""
        ############
        
        rbr_data = get_rbr_data(df2, rbr_id)

        max_weight_code = get_max_weight_code(rbr_data, rbr_id_map)
        
        df_new = pd.concat([df_fix, max_weight_code], axis=1)
        
        exp_code = exp_or_code(choice)
        
        sub_df = df_new[df_new[exp_code].notna()][['Perm ID', 'fiscal_year', exp_code]] # extract non-NaN
        row_labels = get_row_label(df_new)
        exp_table = create_exp_table(df_new, row_labels)
        
        exp_filled = fill_exposure(df_new, exp_table, exp_code) # fill exposure or code table
        exp_filled_fb = front_back_fill(exp_filled) # front and back fill
        exp_timeseries = create_timeseries(exp_filled_fb)

        opid_list = exp_timeseries.columns[1:]
        col_data = get_index_weight(df5_key, exp_timeseries)
        #print(col_data[:10])
        col_name = index_abv_list[i]
        output.loc[:, col_name] = col_data # add a new column with column name equal to the index name
        
    return output   

start = time.time()
# False means not limit to level 1
# df_fix is the first 4 columns of rbr data with perm ID 
# df1 is the rbr map, df2 is the rbr data, and df5 is copied from the rbr selected calc (syntax 3000)
#level23_weight = max_weight_code_name(index_list, False, df_fix, df1, df2, df5, 2)
level1_weight = max_weight_code_name(index_list, True, df_fix, df1, df2, df3, df5, 1)
end = time.time()
print(end - start)

['Syntax Thematic Cybersecurity Index']
[44239]
{'44239': 'Syntax Thematic Cybersecurity Index'}


C:\Users\reaz_\AppData\Local\Temp\ipykernel_9868\2735201883.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5_key['Snapshot Date (Year)'] = df5_key['Snapshot Date (Year)'].astype(int)


['Syntax Thematic Cloud Compute Index']
[46080]
{'46080': 'Syntax Thematic Cloud Compute Index'}
['Syntax Thematic IoT Index']
[46087]
{'46087': 'Syntax Thematic IoT Index'}
['Syntax Thematic E-commerce Index']
[46098]
{'46098': 'Syntax Thematic E-commerce Index'}
['Syntax Thematic Battery Index']
[41753]
{'41753': 'Syntax Thematic Battery Index'}
['Syntax Thematic Bio Revolution Index']
[44209]
{'44209': 'Syntax Thematic Bio Revolution Index'}
['Syntax Thematic Clean Energy Index']
[44217]
{'44217': 'Syntax Thematic Clean Energy Index'}
['Syntax Thematic Software-as-a-Service (SaaS) Index']
[46112]
{'46112': 'Syntax Thematic Software-as-a-Service (SaaS) Index'}
['Syntax Thematic Defensive Index']
[46125]
{'46125': 'Syntax Thematic Defensive Index'}
['Syntax Thematic Infrastructure Index']
[46130]
{'46130': 'Syntax Thematic Infrastructure Index'}
['Syntax Thematic Real Asset Index']
[46144]
{'46144': 'Syntax Thematic Real Asset Index'}
['Syntax Thematic Inflation Index']
[46017]
{'4601

In [12]:
print('The number of unique Perm ID:', df_fix['Perm ID'].nunique())
print('The number of unique Perm ID:', df5['Final ID'].nunique())

a = df_fix['isin'].unique().tolist()
b = df5['ISIN'].unique().tolist()
c = []
for i in b:
    if str(i) not in a:
        c.append(i)
        
level1_exp = level1_weight[~level1_weight['ISIN'].isin(c)]
level1_exp_copy = level1_exp.copy()

The number of unique Perm ID: 11157
The number of unique Perm ID: 2997


In [13]:
def amendments(amend_df, input_df, max_year):
    for index, row in amend_df.iterrows():
        last_yr = row['last year']
        index_name = row['Taxonomy']
        isin = row['Identifier']
        if last_yr == 'nan':
            input_df.loc[input_df['ISIN'] == isin, index_name] = 0
        else:
            year_ls = list(range(int(float(last_yr) + 1), max_year))
            input_df.loc[(input_df['Snapshot Date (Year)'].isin(year_ls)) & (input_df['ISIN'] == isin), index_name] = 0
    
    return input_df


"""Final Cleaning"""
saas_isin = saas[saas['EK SaaS Check'] != 'Y']['isin'].unique()
## df is the exposure or sector code table
amend_df['last year'] = amend_df['last year'].astype(str)
year = amend_df['last year'].unique().tolist()
max_year = level1_exp_copy['Snapshot Date (Year)'].max()
level1_exp_copy.loc[level1_exp_copy['ISIN'].isin(saas_isin), 'SaaS'] = 0
output = amendments(amend_df, level1_exp_copy, max_year)

display(output.head())
output.fillna(0, inplace=True)
output2 = output.loc[~(output[index_abv_list]==0).all(axis=1)]
print(output.shape)
print(output2.shape)

,Snapshot Date,Snapshot Date (Year),Weight Date,Rebal Date,Ex Date,SN3k index weight,OPID,DSCD,ISIN,Final ID,...,E-Commerce,Battery,Bio Revolution,Clean Energy,SaaS,Defensive,Infrastructure,Real Asset,Inflation,Digital Health
0,2023-05-31,2023,2023-06-07,2023-06-16,2023-06-20,0.000039,5074022368.0,938972,US74319R1014,5074022368,...,52.2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2.183482,0.0
1,2023-05-31,2023,2023-06-07,2023-06-16,2023-06-20,0.000011,4295903310.0,923401,US0147521092,4295903310,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.829244,2.379333,0.0
2,2023-05-31,2023,2023-06-07,2023-06-16,2023-06-20,0.000167,5000583878.0,68595V,US74164M1080,5000583878,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
3,2023-05-31,2023,2023-06-07,2023-06-16,2023-06-20,0.000914,4295903261.0,933185,US0010551028,4295903261,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
4,2023-05-31,2023,2023-06-07,2023-06-16,2023-06-20,0.000946,4295903341.0,916305,US0268747849,4295903341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0


(2975, 23)
(1643, 23)


In [14]:
output2.to_excel('exp_level1_March13.xlsx')

In [15]:
# xx = pd.read_excel('C:\\Users\\rzhou\\Downloads\\March13_Thematic_Data\\exp_group_March13.xlsx')

# yy = pd.read_excel('C:\\Users\\rzhou\\Downloads\\March13_Thematic_Data\\weight_date_data.xlsx')
# yy.columns
# xx['DSCD'] = xx['DSCD'].astype(str)
# yy['DSCD'] = yy['DSCD'].astype(str)
# xx2 = xx.merge(yy, on=['DSCD'], how='left')

# display(xx2.head())

# xx2.to_excel('C:\\Users\\rzhou\\Downloads\\March13_Thematic_Data\\exp_group_March13_v2.xlsx')

In [16]:
# df2 = pd.read_excel('C:\\Users\\rzhou\\Downloads\\Project_1\\no_filter_rbr_data_package_2023-03-24_v3.xlsx', 
#                     converters={'gvkey': str})
# df2.loc[df2['isin'] == 'US92537N1081', '46130']